# Module 08: User Authentication

**Estimated Time:** 2.5 hours  
**Difficulty:** Intermediate

---

## Learning Objectives

By the end of this module, you will:

- ✅ Understand Django's authentication system
- ✅ Implement user registration
- ✅ Create login and logout views
- ✅ Handle password reset and change
- ✅ Use `@login_required` decorator
- ✅ Manage user permissions
- ✅ Create user profiles
- ✅ Implement authentication mixins

---

## 1. Introduction to Django Authentication

Django provides a complete authentication system out of the box:

### Built-in Features
- User model (`django.contrib.auth.models.User`)
- Password hashing and validation
- Login/logout views
- Password reset functionality
- Permission and group management
- Session management

### User Model Fields
- `username`: Required, unique
- `password`: Automatically hashed
- `email`: Optional
- `first_name`, `last_name`: Optional
- `is_active`: Boolean, default True
- `is_staff`: Can access admin
- `is_superuser`: All permissions
- `date_joined`: Auto timestamp

In [ ]:
from pathlib import Path

# Setup paths
notebook_dir = Path.cwd()
project_path = notebook_dir.parent / "projects" / "myblog"
blog_app = project_path / "blog"

# Create accounts app for authentication
accounts_app = project_path / "accounts"
accounts_app.mkdir(exist_ok=True)

print(f"Project: {project_path}")
print(f"Accounts app: {accounts_app}")

## 2. Creating the Accounts App

We'll create a separate app for user authentication to keep it modular.

In [ ]:
# Create accounts app structure
(accounts_app / "__init__.py").touch()

# apps.py
apps_code = """from django.apps import AppConfig


class AccountsConfig(AppConfig):
    default_auto_field = 'django.db.models.BigAutoField'
    name = 'accounts'
    
    def ready(self):
        import accounts.signals  # Import signals
"""

with open(accounts_app / "apps.py", "w") as f:
    f.write(apps_code)

# models.py (for UserProfile)
models_code = '''from django.db import models
from django.contrib.auth.models import User
from django.utils.text import slugify


class UserProfile(models.Model):
    """Extended user profile information"""
    user = models.OneToOneField(User, on_delete=models.CASCADE, related_name='profile')
    bio = models.TextField(max_length=500, blank=True)
    avatar = models.ImageField(upload_to='avatars/', blank=True, null=True)
    website = models.URLField(max_length=200, blank=True)
    location = models.CharField(max_length=100, blank=True)
    birth_date = models.DateField(null=True, blank=True)
    
    # Social media
    twitter = models.CharField(max_length=100, blank=True)
    github = models.CharField(max_length=100, blank=True)
    linkedin = models.CharField(max_length=100, blank=True)
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        verbose_name = 'User Profile'
        verbose_name_plural = 'User Profiles'
    
    def __str__(self):
        return f"{self.user.username}'s Profile"
    
    @property
    def full_name(self):
        return f"{self.user.first_name} {self.user.last_name}".strip() or self.user.username
'''

with open(accounts_app / "models.py", "w") as f:
    f.write(models_code)

print("✓ Accounts app created!")
print("\nFiles created:")
print("  - apps.py")
print("  - models.py (UserProfile model)")

## 3. User Profile Signals

Automatically create a UserProfile when a new user registers.

In [ ]:
# Create signals.py
signals_code = '''from django.db.models.signals import post_save
from django.contrib.auth.models import User
from django.dispatch import receiver
from .models import UserProfile


@receiver(post_save, sender=User)
def create_user_profile(sender, instance, created, **kwargs):
    """Create UserProfile when User is created"""
    if created:
        UserProfile.objects.create(user=instance)


@receiver(post_save, sender=User)
def save_user_profile(sender, instance, **kwargs):
    """Save UserProfile when User is saved"""
    if hasattr(instance, 'profile'):
        instance.profile.save()
'''

with open(accounts_app / "signals.py", "w") as f:
    f.write(signals_code)

print("✓ Signals created!")
print("\nSignals:")
print("  - create_user_profile: Auto-create profile on user creation")
print("  - save_user_profile: Auto-save profile when user is saved")

## 4. Registration Forms

Create custom registration and profile forms.

In [ ]:
# Create forms.py
forms_code = '''from django import forms
from django.contrib.auth.forms import UserCreationForm, UserChangeForm
from django.contrib.auth.models import User
from .models import UserProfile


class SignUpForm(UserCreationForm):
    """Custom user registration form"""
    email = forms.EmailField(
        required=True,
        widget=forms.EmailInput(attrs={
            'class': 'form-control',
            'placeholder': 'your@email.com'
        })
    )
    first_name = forms.CharField(
        max_length=150,
        required=True,
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'First Name'
        })
    )
    last_name = forms.CharField(
        max_length=150,
        required=True,
        widget=forms.TextInput(attrs={
            'class': 'form-control',
            'placeholder': 'Last Name'
        })
    )
    
    class Meta:
        model = User
        fields = ['username', 'first_name', 'last_name', 'email', 'password1', 'password2']
        widgets = {
            'username': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'Username'
            }),
        }
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Add form-control class to password fields
        self.fields['password1'].widget.attrs.update({
            'class': 'form-control',
            'placeholder': 'Password'
        })
        self.fields['password2'].widget.attrs.update({
            'class': 'form-control',
            'placeholder': 'Confirm Password'
        })
    
    def clean_email(self):
        email = self.cleaned_data.get('email')
        if User.objects.filter(email=email).exists():
            raise forms.ValidationError('This email is already registered.')
        return email


class UserUpdateForm(forms.ModelForm):
    """Update user information"""
    class Meta:
        model = User
        fields = ['username', 'email', 'first_name', 'last_name']
        widgets = {
            'username': forms.TextInput(attrs={'class': 'form-control'}),
            'email': forms.EmailInput(attrs={'class': 'form-control'}),
            'first_name': forms.TextInput(attrs={'class': 'form-control'}),
            'last_name': forms.TextInput(attrs={'class': 'form-control'}),
        }


class ProfileUpdateForm(forms.ModelForm):
    """Update user profile"""
    class Meta:
        model = UserProfile
        fields = ['bio', 'avatar', 'website', 'location', 'birth_date',
                  'twitter', 'github', 'linkedin']
        widgets = {
            'bio': forms.Textarea(attrs={
                'class': 'form-control',
                'rows': 4,
                'placeholder': 'Tell us about yourself...'
            }),
            'website': forms.URLInput(attrs={'class': 'form-control'}),
            'location': forms.TextInput(attrs={'class': 'form-control'}),
            'birth_date': forms.DateInput(attrs={
                'class': 'form-control',
                'type': 'date'
            }),
            'twitter': forms.TextInput(attrs={'class': 'form-control', 'placeholder': '@username'}),
            'github': forms.TextInput(attrs={'class': 'form-control', 'placeholder': 'username'}),
            'linkedin': forms.TextInput(attrs={'class': 'form-control', 'placeholder': 'username'}),
        }
'''

with open(accounts_app / "forms.py", "w") as f:
    f.write(forms_code)

print("✓ Forms created!")
print("\nForms:")
print("  - SignUpForm: User registration with email validation")
print("  - UserUpdateForm: Update user details")
print("  - ProfileUpdateForm: Update profile information")

## 5. Authentication Views

Create views for registration, login, logout, and profile management.

In [ ]:
# Create views.py
views_code = '''from django.shortcuts import render, redirect
from django.contrib import messages
from django.contrib.auth import login, authenticate, logout
from django.contrib.auth.decorators import login_required
from django.contrib.auth.views import (
    LoginView, LogoutView, PasswordResetView, PasswordChangeView
)
from django.urls import reverse_lazy
from django.views.generic import CreateView, UpdateView, DetailView
from django.contrib.auth.models import User
from .forms import SignUpForm, UserUpdateForm, ProfileUpdateForm
from .models import UserProfile


# Function-based views
def signup_view(request):
    """User registration view"""
    if request.user.is_authenticated:
        return redirect('blog:home')
    
    if request.method == 'POST':
        form = SignUpForm(request.POST)
        if form.is_valid():
            user = form.save()
            # Log the user in
            username = form.cleaned_data.get('username')
            password = form.cleaned_data.get('password1')
            user = authenticate(username=username, password=password)
            login(request, user)
            messages.success(request, f'Welcome {username}! Your account has been created.')
            return redirect('blog:home')
    else:
        form = SignUpForm()
    
    return render(request, 'accounts/signup.html', {'form': form})


@login_required
def profile_view(request):
    """View user profile"""
    return render(request, 'accounts/profile.html', {
        'user': request.user,
        'profile': request.user.profile
    })


@login_required
def profile_edit_view(request):
    """Edit user profile"""
    if request.method == 'POST':
        user_form = UserUpdateForm(request.POST, instance=request.user)
        profile_form = ProfileUpdateForm(
            request.POST,
            request.FILES,
            instance=request.user.profile
        )
        
        if user_form.is_valid() and profile_form.is_valid():
            user_form.save()
            profile_form.save()
            messages.success(request, 'Your profile has been updated!')
            return redirect('accounts:profile')
    else:
        user_form = UserUpdateForm(instance=request.user)
        profile_form = ProfileUpdateForm(instance=request.user.profile)
    
    context = {
        'user_form': user_form,
        'profile_form': profile_form
    }
    return render(request, 'accounts/profile_edit.html', context)


# Class-based views
class CustomLoginView(LoginView):
    """Custom login view"""
    template_name = 'accounts/login.html'
    redirect_authenticated_user = True
    
    def get_success_url(self):
        return reverse_lazy('blog:home')
    
    def form_valid(self, form):
        messages.success(self.request, f'Welcome back, {form.get_user().username}!')
        return super().form_valid(form)


class CustomLogoutView(LogoutView):
    """Custom logout view"""
    next_page = reverse_lazy('blog:home')
    
    def dispatch(self, request, *args, **kwargs):
        if request.user.is_authenticated:
            messages.info(request, 'You have been logged out.')
        return super().dispatch(request, *args, **kwargs)


class UserProfileView(DetailView):
    """Public user profile view"""
    model = User
    template_name = 'accounts/user_profile.html'
    context_object_name = 'profile_user'
    slug_field = 'username'
    slug_url_kwarg = 'username'
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        # Get user's posts
        from blog.models import Post
        context['posts'] = Post.objects.filter(
            author=self.object,
            status='published'
        ).order_by('-publish_date')[:5]
        return context
'''

with open(accounts_app / "views.py", "w") as f:
    f.write(views_code)

print("✓ Views created!")
print("\nViews:")
print("  - signup_view: User registration (FBV)")
print("  - profile_view: View own profile (FBV)")
print("  - profile_edit_view: Edit profile (FBV)")
print("  - CustomLoginView: Login (CBV)")
print("  - CustomLogoutView: Logout (CBV)")
print("  - UserProfileView: Public profile (CBV)")

## 6. URL Configuration

In [ ]:
# Create accounts URLs
urls_code = """from django.urls import path
from django.contrib.auth import views as auth_views
from . import views

app_name = 'accounts'

urlpatterns = [
    # Registration
    path('signup/', views.signup_view, name='signup'),
    
    # Login/Logout
    path('login/', views.CustomLoginView.as_view(), name='login'),
    path('logout/', views.CustomLogoutView.as_view(), name='logout'),
    
    # Profile
    path('profile/', views.profile_view, name='profile'),
    path('profile/edit/', views.profile_edit_view, name='profile_edit'),
    path('user/<str:username>/', views.UserProfileView.as_view(), name='user_profile'),
    
    # Password management
    path('password-change/', 
         auth_views.PasswordChangeView.as_view(
             template_name='accounts/password_change.html',
             success_url='/accounts/password-change/done/'
         ), 
         name='password_change'),
    path('password-change/done/', 
         auth_views.PasswordChangeDoneView.as_view(
             template_name='accounts/password_change_done.html'
         ), 
         name='password_change_done'),
    
    # Password reset
    path('password-reset/', 
         auth_views.PasswordResetView.as_view(
             template_name='accounts/password_reset.html',
             email_template_name='accounts/password_reset_email.html',
             success_url='/accounts/password-reset/done/'
         ), 
         name='password_reset'),
    path('password-reset/done/', 
         auth_views.PasswordResetDoneView.as_view(
             template_name='accounts/password_reset_done.html'
         ), 
         name='password_reset_done'),
    path('password-reset-confirm/<uidb64>/<token>/', 
         auth_views.PasswordResetConfirmView.as_view(
             template_name='accounts/password_reset_confirm.html',
             success_url='/accounts/password-reset-complete/'
         ), 
         name='password_reset_confirm'),
    path('password-reset-complete/', 
         auth_views.PasswordResetCompleteView.as_view(
             template_name='accounts/password_reset_complete.html'
         ), 
         name='password_reset_complete'),
]
"""

with open(accounts_app / "urls.py", "w") as f:
    f.write(urls_code)

print("✓ URLs created!")
print("\nURL patterns:")
print("  - /signup/: User registration")
print("  - /login/: User login")
print("  - /logout/: User logout")
print("  - /profile/: Own profile")
print("  - /profile/edit/: Edit profile")
print("  - /user/<username>/: Public profile")
print("  - /password-change/: Change password")
print("  - /password-reset/: Reset password via email")

## 7. Create Templates

Let's create authentication templates.

In [ ]:
# Create templates directory
templates_dir = accounts_app / "templates" / "accounts"
templates_dir.mkdir(parents=True, exist_ok=True)

# Signup template
signup_template = """{% extends 'blog/base.html' %}

{% block title %}Sign Up{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Create an Account</h2>
    
    <form method="post" class="auth-form">
        {% csrf_token %}
        
        {% if form.non_field_errors %}
        <div class="errors">
            {{ form.non_field_errors }}
        </div>
        {% endif %}
        
        {% for field in form %}
        <div class="form-group">
            {{ field.label_tag }}
            {{ field }}
            {% if field.help_text %}
            <small class="form-text">{{ field.help_text|safe }}</small>
            {% endif %}
            {% if field.errors %}
            <div class="field-errors">
                {{ field.errors }}
            </div>
            {% endif %}
        </div>
        {% endfor %}
        
        <button type="submit" class="btn btn-primary">Sign Up</button>
    </form>
    
    <p class="auth-link">
        Already have an account? <a href="{% url 'accounts:login' %}">Log in</a>
    </p>
</div>

<style>
.auth-container {
    max-width: 500px;
    margin: 2rem auto;
    padding: 2rem;
    background: white;
    border-radius: 8px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.auth-link {
    text-align: center;
    margin-top: 1rem;
}
</style>
{% endblock %}
"""

with open(templates_dir / "signup.html", "w") as f:
    f.write(signup_template)

# Login template
login_template = """{% extends 'blog/base.html' %}

{% block title %}Login{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Login</h2>
    
    <form method="post" class="auth-form">
        {% csrf_token %}
        
        {% if form.non_field_errors %}
        <div class="errors">
            {{ form.non_field_errors }}
        </div>
        {% endif %}
        
        <div class="form-group">
            {{ form.username.label_tag }}
            {{ form.username }}
            {% if form.username.errors %}
            <div class="field-errors">{{ form.username.errors }}</div>
            {% endif %}
        </div>
        
        <div class="form-group">
            {{ form.password.label_tag }}
            {{ form.password }}
            {% if form.password.errors %}
            <div class="field-errors">{{ form.password.errors }}</div>
            {% endif %}
        </div>
        
        <button type="submit" class="btn btn-primary">Login</button>
        
        <p class="auth-link">
            <a href="{% url 'accounts:password_reset' %}">Forgot password?</a>
        </p>
    </form>
    
    <p class="auth-link">
        Don\'t have an account? <a href="{% url 'accounts:signup' %}">Sign up</a>
    </p>
</div>
{% endblock %}
"""

with open(templates_dir / "login.html", "w") as f:
    f.write(login_template)

# Profile template
profile_template = """{% extends 'blog/base.html' %}

{% block title %}{{ user.username }}\'s Profile{% endblock %}

{% block content %}
<div class="profile-container">
    <div class="profile-header">
        {% if profile.avatar %}
        <img src="{{ profile.avatar.url }}" alt="Avatar" class="avatar-large">
        {% else %}
        <div class="avatar-placeholder">{{ user.username.0|upper }}</div>
        {% endif %}
        
        <div class="profile-info">
            <h2>{{ profile.full_name }}</h2>
            <p class="username">@{{ user.username }}</p>
            
            {% if profile.bio %}
            <p class="bio">{{ profile.bio }}</p>
            {% endif %}
            
            <div class="profile-meta">
                {% if profile.location %}
                <span>📍 {{ profile.location }}</span>
                {% endif %}
                {% if profile.website %}
                <span>🔗 <a href="{{ profile.website }}" target="_blank">Website</a></span>
                {% endif %}
                <span>📅 Joined {{ user.date_joined|date:"F Y" }}</span>
            </div>
            
            {% if profile.twitter or profile.github or profile.linkedin %}
            <div class="social-links">
                {% if profile.twitter %}
                <a href="https://twitter.com/{{ profile.twitter }}" target="_blank">Twitter</a>
                {% endif %}
                {% if profile.github %}
                <a href="https://github.com/{{ profile.github }}" target="_blank">GitHub</a>
                {% endif %}
                {% if profile.linkedin %}
                <a href="https://linkedin.com/in/{{ profile.linkedin }}" target="_blank">LinkedIn</a>
                {% endif %}
            </div>
            {% endif %}
            
            <div class="profile-actions">
                <a href="{% url 'accounts:profile_edit' %}" class="btn">Edit Profile</a>
                <a href="{% url 'accounts:password_change' %}" class="btn">Change Password</a>
            </div>
        </div>
    </div>
</div>

<style>
.profile-container {
    max-width: 800px;
    margin: 2rem auto;
}
.profile-header {
    display: flex;
    gap: 2rem;
    padding: 2rem;
    background: white;
    border-radius: 8px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.avatar-large {
    width: 150px;
    height: 150px;
    border-radius: 50%;
    object-fit: cover;
}
.avatar-placeholder {
    width: 150px;
    height: 150px;
    border-radius: 50%;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    display: flex;
    align-items: center;
    justify-content: center;
    font-size: 4rem;
    font-weight: bold;
}
.profile-info {
    flex: 1;
}
.username {
    color: #666;
    margin-bottom: 1rem;
}
.bio {
    margin: 1rem 0;
    line-height: 1.6;
}
.profile-meta {
    display: flex;
    gap: 1rem;
    flex-wrap: wrap;
    margin: 1rem 0;
    color: #666;
    font-size: 0.9rem;
}
.social-links {
    display: flex;
    gap: 1rem;
    margin: 1rem 0;
}
.profile-actions {
    display: flex;
    gap: 0.5rem;
    margin-top: 1.5rem;
}
</style>
{% endblock %}
"""

with open(templates_dir / "profile.html", "w") as f:
    f.write(profile_template)

print("✓ Templates created!")
print("\nTemplates:")
print("  - signup.html")
print("  - login.html")
print("  - profile.html")

In [ ]:
# Profile edit template
profile_edit_template = """{% extends 'blog/base.html' %}

{% block title %}Edit Profile{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Edit Profile</h2>
    
    <form method="post" enctype="multipart/form-data">
        {% csrf_token %}
        
        <h3>Account Information</h3>
        {% for field in user_form %}
        <div class="form-group">
            {{ field.label_tag }}
            {{ field }}
            {% if field.errors %}
            <div class="field-errors">{{ field.errors }}</div>
            {% endif %}
        </div>
        {% endfor %}
        
        <h3>Profile Details</h3>
        {% for field in profile_form %}
        <div class="form-group">
            {{ field.label_tag }}
            {{ field }}
            {% if field.help_text %}
            <small>{{ field.help_text }}</small>
            {% endif %}
            {% if field.errors %}
            <div class="field-errors">{{ field.errors }}</div>
            {% endif %}
        </div>
        {% endfor %}
        
        <div class="form-actions">
            <button type="submit" class="btn btn-primary">Save Changes</button>
            <a href="{% url 'accounts:profile' %}" class="btn">Cancel</a>
        </div>
    </form>
</div>
{% endblock %}
"""

with open(templates_dir / "profile_edit.html", "w") as f:
    f.write(profile_edit_template)

# Password change template
password_change_template = """{% extends 'blog/base.html' %}

{% block title %}Change Password{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Change Password</h2>
    
    <form method="post">
        {% csrf_token %}
        
        {% for field in form %}
        <div class="form-group">
            {{ field.label_tag }}
            {{ field }}
            {% if field.help_text %}
            <small>{{ field.help_text|safe }}</small>
            {% endif %}
            {% if field.errors %}
            <div class="field-errors">{{ field.errors }}</div>
            {% endif %}
        </div>
        {% endfor %}
        
        <button type="submit" class="btn btn-primary">Change Password</button>
    </form>
</div>
{% endblock %}
"""

with open(templates_dir / "password_change.html", "w") as f:
    f.write(password_change_template)

# Password change done
password_change_done_template = """{% extends 'blog/base.html' %}

{% block title %}Password Changed{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Password Changed Successfully</h2>
    <p>Your password has been changed.</p>
    <a href="{% url 'accounts:profile' %}" class="btn">Back to Profile</a>
</div>
{% endblock %}
"""

with open(templates_dir / "password_change_done.html", "w") as f:
    f.write(password_change_done_template)

# Password reset template
password_reset_template = """{% extends 'blog/base.html' %}

{% block title %}Reset Password{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Reset Password</h2>
    <p>Enter your email address to receive password reset instructions.</p>
    
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit" class="btn btn-primary">Send Reset Email</button>
    </form>
</div>
{% endblock %}
"""

with open(templates_dir / "password_reset.html", "w") as f:
    f.write(password_reset_template)

# Password reset email
password_reset_email = """Hi {{ user.username }},

You requested a password reset for your account at {{ site_name }}.

Click the link below to reset your password:
{{ protocol }}://{{ domain }}{% url 'accounts:password_reset_confirm' uidb64=uid token=token %}

If you didn\'t request this, please ignore this email.

Thanks,
The {{ site_name }} Team
"""

with open(templates_dir / "password_reset_email.html", "w") as f:
    f.write(password_reset_email)

# Password reset done
password_reset_done_template = """{% extends 'blog/base.html' %}

{% block title %}Password Reset Sent{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Password Reset Email Sent</h2>
    <p>We\'ve sent you instructions for resetting your password. Check your email.</p>
    <a href="{% url 'accounts:login' %}" class="btn">Back to Login</a>
</div>
{% endblock %}
"""

with open(templates_dir / "password_reset_done.html", "w") as f:
    f.write(password_reset_done_template)

# Password reset confirm
password_reset_confirm_template = """{% extends 'blog/base.html' %}

{% block title %}Confirm Password Reset{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Enter New Password</h2>
    
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit" class="btn btn-primary">Reset Password</button>
    </form>
</div>
{% endblock %}
"""

with open(templates_dir / "password_reset_confirm.html", "w") as f:
    f.write(password_reset_confirm_template)

# Password reset complete
password_reset_complete_template = """{% extends 'blog/base.html' %}

{% block title %}Password Reset Complete{% endblock %}

{% block content %}
<div class="auth-container">
    <h2>Password Reset Complete</h2>
    <p>Your password has been reset. You can now log in with your new password.</p>
    <a href="{% url 'accounts:login' %}" class="btn btn-primary">Log In</a>
</div>
{% endblock %}
"""

with open(templates_dir / "password_reset_complete.html", "w") as f:
    f.write(password_reset_complete_template)

print("✓ All authentication templates created!")
print("\nAdditional templates:")
print("  - profile_edit.html")
print("  - password_change.html")
print("  - password_change_done.html")
print("  - password_reset.html")
print("  - password_reset_email.html")
print("  - password_reset_done.html")
print("  - password_reset_confirm.html")
print("  - password_reset_complete.html")

## 8. Update Base Template

Add authentication links to the navigation.

In [ ]:
# Note: You would update blog/templates/blog/base.html manually
# Add to navigation:

updated_nav = """
<nav>
    <a href="{% url 'blog:home' %}">Home</a>
    <a href="{% url 'blog:about' %}">About</a>
    
    {% if user.is_authenticated %}
        <a href="{% url 'blog:post_create' %}">New Post</a>
        <a href="{% url 'accounts:profile' %}">Profile</a>
        <a href="{% url 'accounts:logout' %}">Logout</a>
        <span class="username">{{ user.username }}</span>
    {% else %}
        <a href="{% url 'accounts:login' %}">Login</a>
        <a href="{% url 'accounts:signup' %}">Sign Up</a>
    {% endif %}
</nav>
"""

print("Navigation code to add to base.html:")
print(updated_nav)
print("\n✓ This shows authenticated/unauthenticated states")

## 9. Protecting Views with Decorators

### @login_required Decorator

```python
from django.contrib.auth.decorators import login_required

@login_required
def my_view(request):
    # Only logged-in users can access
    pass
```

### LoginRequiredMixin for CBVs

```python
from django.contrib.auth.mixins import LoginRequiredMixin

class MyView(LoginRequiredMixin, View):
    login_url = '/accounts/login/'
    redirect_field_name = 'next'
```

### Permission Required

```python
from django.contrib.auth.decorators import permission_required

@permission_required('blog.add_post')
def create_post(request):
    pass
```

## 10. Update Post Views with Authentication

Protect post creation/editing views.

In [ ]:
# Example: Update PostCreateView
protected_view_example = """
from django.contrib.auth.mixins import LoginRequiredMixin

class PostCreateView(LoginRequiredMixin, CreateView):
    model = Post
    form_class = PostForm
    template_name = 'blog/post_form.html'
    login_url = '/accounts/login/'
    
    def form_valid(self, form):
        form.instance.author = self.request.user  # Auto-set author
        return super().form_valid(form)


class PostUpdateView(LoginRequiredMixin, UpdateView):
    model = Post
    form_class = PostForm
    template_name = 'blog/post_form.html'
    
    def get_queryset(self):
        # Users can only edit their own posts
        return Post.objects.filter(author=self.request.user)
"""

print("Example of protected views:")
print(protected_view_example)
print("\n✓ Only authenticated users can create/edit posts")
print("✓ Users can only edit their own posts")

## 11. User Permissions & Groups

### Checking Permissions in Templates

```django
{% if perms.blog.add_post %}
    <a href="{% url 'blog:post_create' %}">Create Post</a>
{% endif %}

{% if user.is_staff %}
    <a href="{% url 'admin:index' %}">Admin</a>
{% endif %}
```

### Checking Permissions in Views

```python
if request.user.has_perm('blog.delete_post'):
    # Allow deletion
    pass

if request.user.is_superuser:
    # Full access
    pass
```

### Creating Groups Programmatically

```python
from django.contrib.auth.models import Group, Permission

# Create group
editors = Group.objects.create(name='Editors')

# Add permissions
perm = Permission.objects.get(codename='add_post')
editors.permissions.add(perm)

# Add user to group
user.groups.add(editors)
```

## 12. Settings Configuration

Add authentication settings to `settings.py`:

In [ ]:
settings_additions = """
# Authentication settings
LOGIN_URL = '/accounts/login/'
LOGIN_REDIRECT_URL = '/'
LOGOUT_REDIRECT_URL = '/'

# Password validation
AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
        'OPTIONS': {
            'min_length': 8,
        }
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]

# Email backend (for development)
EMAIL_BACKEND = 'django.core.mail.backends.console.EmailBackend'

# For production, use real email:
# EMAIL_BACKEND = 'django.core.mail.backends.smtp.EmailBackend'
# EMAIL_HOST = 'smtp.gmail.com'
# EMAIL_PORT = 587
# EMAIL_USE_TLS = True
# EMAIL_HOST_USER = 'your-email@gmail.com'
# EMAIL_HOST_PASSWORD = 'your-password'
"""

print("Add to settings.py:")
print(settings_additions)

## 13. Admin Configuration

In [ ]:
# Create admin.py
admin_code = '''from django.contrib import admin
from django.contrib.auth.admin import UserAdmin as BaseUserAdmin
from django.contrib.auth.models import User
from .models import UserProfile


class UserProfileInline(admin.StackedInline):
    """Inline admin for UserProfile"""
    model = UserProfile
    can_delete = False
    verbose_name_plural = 'Profile'
    fk_name = 'user'


class UserAdmin(BaseUserAdmin):
    """Custom User admin with profile inline"""
    inlines = (UserProfileInline,)
    list_display = ['username', 'email', 'first_name', 'last_name', 'is_staff', 'date_joined']
    list_filter = ['is_staff', 'is_superuser', 'is_active', 'groups']


# Re-register UserAdmin
admin.site.unregister(User)
admin.site.register(User, UserAdmin)


@admin.register(UserProfile)
class UserProfileAdmin(admin.ModelAdmin):
    list_display = ['user', 'location', 'created_at']
    search_fields = ['user__username', 'user__email', 'location']
    list_filter = ['created_at']
'''

with open(accounts_app / "admin.py", "w") as f:
    f.write(admin_code)

print("✓ Admin configuration created!")
print("\nAdmin features:")
print("  - UserProfile inline in User admin")
print("  - Extended user list display")
print("  - Profile search and filtering")

## 14. Security Best Practices

### Password Security
1. **Never store plain text passwords** - Django handles hashing
2. **Use strong password validators** - Already configured in settings
3. **Implement password reset** - Email-based reset flow
4. **Set password expiry** - For high-security apps

### Session Security
```python
# settings.py
SESSION_COOKIE_SECURE = True  # HTTPS only
SESSION_COOKIE_HTTPONLY = True  # No JavaScript access
SESSION_COOKIE_SAMESITE = 'Strict'  # CSRF protection
SESSION_COOKIE_AGE = 1209600  # 2 weeks
```

### User Input Validation
1. Always validate and sanitize user input
2. Use Django's built-in validators
3. Implement CSRF protection (automatic)
4. Escape HTML in templates (automatic)

### Account Security
1. **Email verification** - Confirm email on signup
2. **Two-factor authentication** - Use django-otp
3. **Rate limiting** - Prevent brute force attacks
4. **Account lockout** - After failed login attempts

## 15. Testing Authentication

In [ ]:
# Create tests.py
tests_code = '''from django.test import TestCase, Client
from django.contrib.auth.models import User
from django.urls import reverse
from .models import UserProfile


class AuthenticationTestCase(TestCase):
    """Test authentication functionality"""
    
    def setUp(self):
        self.client = Client()
        self.user = User.objects.create_user(
            username='testuser',
            email='test@example.com',
            password='testpass123'
        )
    
    def test_signup(self):
        """Test user registration"""
        response = self.client.post(reverse('accounts:signup'), {
            'username': 'newuser',
            'email': 'new@example.com',
            'first_name': 'New',
            'last_name': 'User',
            'password1': 'complexpass123',
            'password2': 'complexpass123',
        })
        self.assertEqual(User.objects.count(), 2)
        self.assertTrue(User.objects.filter(username='newuser').exists())
    
    def test_login(self):
        """Test user login"""
        response = self.client.post(reverse('accounts:login'), {
            'username': 'testuser',
            'password': 'testpass123',
        })
        self.assertEqual(response.status_code, 302)  # Redirect
        self.assertTrue(response.wsgi_request.user.is_authenticated)
    
    def test_logout(self):
        """Test user logout"""
        self.client.login(username='testuser', password='testpass123')
        response = self.client.post(reverse('accounts:logout'))
        self.assertFalse(response.wsgi_request.user.is_authenticated)
    
    def test_profile_creation(self):
        """Test UserProfile auto-creation"""
        self.assertTrue(hasattr(self.user, 'profile'))
        self.assertIsInstance(self.user.profile, UserProfile)
    
    def test_login_required(self):
        """Test @login_required decorator"""
        response = self.client.get(reverse('accounts:profile'))
        self.assertEqual(response.status_code, 302)  # Redirect to login
        
        # Login and try again
        self.client.login(username='testuser', password='testpass123')
        response = self.client.get(reverse('accounts:profile'))
        self.assertEqual(response.status_code, 200)  # Success
'''

with open(accounts_app / "tests.py", "w") as f:
    f.write(tests_code)

print("✓ Tests created!")
print("\nTest cases:")
print("  - test_signup: User registration")
print("  - test_login: User login")
print("  - test_logout: User logout")
print("  - test_profile_creation: Auto profile creation")
print("  - test_login_required: Protected views")
print("\nRun tests with: python manage.py test accounts")

## 16. Summary & Next Steps

### What We Accomplished

✅ Created accounts app with UserProfile model  
✅ Implemented user registration with custom form  
✅ Built login/logout functionality  
✅ Created profile views and edit forms  
✅ Implemented password change/reset flows  
✅ Added authentication decorators  
✅ Protected views with LoginRequiredMixin  
✅ Created comprehensive templates  
✅ Configured admin for user management  
✅ Implemented security best practices  
✅ Wrote authentication tests  

### Key Concepts Learned

1. **Django Auth System**: Built-in User model and authentication
2. **User Registration**: Custom signup forms with validation
3. **Login/Logout**: Session-based authentication
4. **Password Management**: Change and reset workflows
5. **User Profiles**: Extending User model with OneToOne
6. **Signals**: Auto-create profiles on user creation
7. **Decorators**: @login_required for function views
8. **Mixins**: LoginRequiredMixin for class views
9. **Permissions**: has_perm(), is_staff, is_superuser
10. **Security**: Password hashing, CSRF, session management

### Files Created

**accounts/** app:
- `models.py`: UserProfile model
- `forms.py`: SignUpForm, UserUpdateForm, ProfileUpdateForm
- `views.py`: Registration, login, profile views
- `urls.py`: Authentication URLs
- `signals.py`: Auto-create profiles
- `admin.py`: Custom User admin
- `tests.py`: Authentication tests

**templates/accounts/**:
- `signup.html`, `login.html`
- `profile.html`, `profile_edit.html`
- `password_change.html`, `password_change_done.html`
- `password_reset.html`, `password_reset_done.html`
- `password_reset_confirm.html`, `password_reset_complete.html`
- `password_reset_email.html`

### What's Next

In **Module 09**, we'll:
- Prepare Django for production deployment
- Configure environment variables
- Implement security checklist
- Set up static file serving
- Configure database for production
- Learn deployment best practices

---

**Excellent work! Your blog now has a complete authentication system. Continue to Module 09!** 🔐